In [7]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import pandas as pd
import re
from collections import defaultdict
from math import log, exp
import nltk
from math import *




In [8]:
# Load the dataset
spam_data = pd.read_csv('spam (2).csv',encoding = 'latin-1')

# Display the first few rows of the dataframe
display(spam_data.head())

# Display basic information about the dataset
print(spam_data.info())

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB
None


In [9]:
data= spam_data.drop_duplicates(keep='first')
data.duplicated().sum()

0

In [10]:
def clean_text(text):
    clean = re.sub('[^a-zA-Z]', ' ', text) # Replacing all non-alphabetic characters with a space
    clean = clean.lower() # converting to lowecase
    clean = clean.split() # splits the cleaned text sms into a list of words
    clean = ' '.join(clean) # joins the list of words back into a string, using a single space as the separator between the words
    return clean

data.loc[:, "Clean_Text"] = data["v2"].apply(clean_text)

C:\Users\Lithish r\AppData\Local\Temp\ipykernel_45828\2052052726.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "Clean_Text"] = data["v2"].apply(clean_text)


In [11]:
messages = spam_data['v2']
messages

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [12]:
labels = spam_data['v1']
labels

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [13]:

class BayesianSpamFilter:
    def __init__(self):
        self.spam_word_counts = defaultdict(int)
        self.ham_word_counts = defaultdict(int)
        self.total_spam_words = 0
        self.total_ham_words = 0
        self.total_spam_messages = 0
        self.total_ham_messages = 0

    def train(self, messages, labels):
        for message, label in zip(messages, labels):
            words = self.extract_words(message)
            if label == 'spam':
                self.total_spam_messages += 1
                for word in words:
                    self.spam_word_counts[word] += 1
                    self.total_spam_words += 1
            else:
                self.total_ham_messages += 1
                for word in words:
                    self.ham_word_counts[word] += 1
                    self.total_ham_words += 1

    def extract_words(self, message):
        return re.findall(r'\b\w+\b', message.lower())

    def calculate_word_probabilities(self, message):
        words = self.extract_words(message)
        log_spam_probability = log(self.total_spam_messages / (self.total_spam_messages + self.total_ham_messages))
        log_ham_probability = log(self.total_ham_messages / (self.total_spam_messages + self.total_ham_messages))
        for word in words:
            # Laplace smoothing
            spam_word_probability = (self.spam_word_counts[word] + 1) / (self.total_spam_words + 2)
            ham_word_probability = (self.ham_word_counts[word] + 1) / (self.total_ham_words + 2)
            log_spam_probability += log(spam_word_probability)
            log_ham_probability += log(ham_word_probability)
        return log_spam_probability, log_ham_probability

    def classify(self, message):
        log_spam_probability, log_ham_probability = self.calculate_word_probabilities(message)

        probability_difference = log_spam_probability - log_ham_probability
        spam_probability = 1 / (1 + exp(-probability_difference))
        return spam_probability
filter = BayesianSpamFilter()
filter.train(messages, labels)
test_message = "Free entry in 2 a wkly comp"
spam_probability = filter.classify(test_message)
print("Spam probability:", spam_probability)






Spam probability: 0.9999942104611959


In [14]:
import re

class HeuristicSpamFilter:
    def __init__(self):
        self.suspicious_keywords = ['free', 'buy now', 'winner', 'guaranteed', 'act now']
        self.spammy_patterns = [r'\b\d{4,}\b']  # Example: 4 or more consecutive digits
        # Adding a simple regex pattern to identify URLs; this can be enhanced for more robust detection
        self.url_pattern = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'

    def check_keywords(self, message):
        for keyword in self.suspicious_keywords:
            if keyword in message.lower():
                return True
        return False

    def check_structure(self, message):
        for pattern in self.spammy_patterns:
            if re.search(pattern, message):
                return True
        return False

    def check_urls(self, message):
        return re.search(self.url_pattern, message) is not None

    def is_spam(self, message):
        if self.check_keywords(message):
            return 'spam'
        if self.check_structure(message):
            return 'spam'
        if self.check_urls(message):
            return 'spam'
        return 'ham'

In [15]:
filterh = HeuristicSpamFilter()

# Apply the filter to classify each SMS message
spam = messages.apply(filterh.is_spam)
accuracy = (spam == labels).mean()
# Display the classified messages
print(accuracy )

0.970208183776023


In [16]:
import pandas as pd

class CollaborativeSpamFilter:
    def __init__(self):
        self.spam_reports = {}  # Dictionary to store spam reports for each message

    def report_spam(self, message):
        if message in self.spam_reports:
            self.spam_reports[message] += 1
        else:
            self.spam_reports[message] = 1

    def is_spam(self, message, threshold=3):
        if message in self.spam_reports:
            if self.spam_reports[message] >= threshold:
                return 'spam'
            else:
                return 'ham'
        return 'ham'

# Example usage
filter = CollaborativeSpamFilter()

# Load the SMS spam dataset  # Update with the actual filename and path

# Choose messages labeled as spam to add to the report
spam_messages = messages[labels == 'spam']

# Simulate user feedback by reporting spam messages
for message in spam_messages:
    filter.report_spam(message)

# Apply the filter to classify messages
isspam = messages.apply(lambda message: filter.is_spam(message))

# Display the classified messages
print(isspam)

0       ham
1       ham
2       ham
3       ham
4       ham
       ... 
5567    ham
5568    ham
5569    ham
5570    ham
5571    ham
Name: v2, Length: 5572, dtype: object


In [17]:
accuracy = (isspam == labels).mean()
# Display the classified messages
print(accuracy )


0.8693467336683417
